In [5]:
import cv2

In [6]:
import numpy as np

In [7]:
import matplotlib.pyplot as pl
pl.ion()

In [12]:
cap = cv2.VideoCapture('vlc-record-2018-04-01-19h35m54s-GP017447.MP4-.mp4')

### Opening video file and saving frames
#### The program ends either when end of file is reached or when user interupts by pressing 'q'

In [6]:
cap = cv2.VideoCapture('vlc-record-2018-04-01-19h35m54s-GP017447.MP4-.mp4')
count = 0 #counting number of frames
while cap.isOpened():
    ret,frame = cap.read() #read a frame
    try:
        cv2.imshow('Frame',frame)
        cv2.imwrite("videoframes/frame%d.jpg" % count, frame)
        count = count + 1
    except:
        print('EOF at frame {}'.format(count))
        break
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        print ("File interupted by user at: {}".format(count))
        break


cap.release()
cv2.destroyAllWindows()

### Drawing in the video frame

In [18]:
cap = cv2.VideoCapture('vlc-record-2018-04-02-14h43m33s-GP017447.MP4-.mp4')
width = cap.get(3) #get width
height = cap.get(4) #get height

mx_coord = int(width/2)
my_coord = int(height/2)

count = 0 # counting number of frames

while cap.isOpened():
    ret, frame = cap.read() # read a frame
    try:
        count = count + 1
        text = "Hello Frame: " + str(count) #Text in the original video frame
        cv2.putText(frame, text, (mx_coord,my_coord), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1,cv2.LINE_AA)
        cv2.imshow('Original',frame)
        frame2 = frame
    except:
        print("File ended at frame: {}".format(count))
        break
    #drawing line on the frame
    line1 = np.array([[100,100],[300,100],[350,200]], np.int32).reshape((-1,1,2))
    
    frame2 = cv2.polylines(frame,[line1],False,(255,255,255),thickness=2)
    cv2.imshow('Frame with Line',frame2)
   
    if cv2.waitKey(10) & 0xFF == ord('q'):
        print ("File interupted by user at: {}".format(count))
        break

cap.release()
cv2.destroyAllWindows()

File ended at frame: 371


### Background subtraction

In [41]:
cap = cv2.VideoCapture('vlc-record-2018-04-02-14h43m33s-GP017447.MP4-.mp4')

subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=True) # Create Backgrounf subtractor

while cap.isOpened():
    ret, frame = cap.read() # read a frame
    
    subtracted_frame = subtractor.apply(frame)
    
    try:
        cv2.imshow('Original Frame', frame)
        cv2.imshow('Subtracted Frame', subtracted_frame)
        
    except:
        print("File ended at frame: {}".format(count))
        break
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        print ("File interupted by user at: {}".format(count))
        break

cap.release()
cv2.destroyAllWindows()

File interupted by user at: 371


### Contour detection

In [9]:
cap = cv2.VideoCapture('vlc-record-2018-04-02-14h43m33s-GP017447.MP4-.mp4')

subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=True) # Create Backgrounf subtractor

kernelOp = np.ones((3,3),np.uint8)
kernelCl = np.ones((11,11),np.uint8)
areaTH = 500

while cap.isOpened():
    ret, frame = cap.read() # read a frame
    
    subtracted_frame = subtractor.apply(frame)
    
    try:
        ret, binary_image = cv2.threshold(subtracted_frame,200,255,cv2.THRESH_BINARY) # converting to binary image
        image_mask = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernelOp) # remove noise
        image_mask = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernelCl) #to join white regions
        
    except:
        print("File ended at frame: {}".format(count))
        break
    
    _, contours0, hierarchy = cv2.findContours(image_mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    for cnt in contours0:
        #cv2.drawContours(frame, cnt, -1, (0,255,0), 3, 8)
        area = cv2.contourArea(cnt)
        print (area)
        if area > areaTH:           
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.circle(frame,(cx,cy), 5, (0,0,255), -1)            
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.imshow('Frame',frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        print("EOF")
        break

cap.release()
cv2.destroyAllWindows()

0.0
8.5
0.0
0.0
0.0
8.5
1.0
0.0
0.0
0.0
1.0
86.5
0.0
0.5
0.0
71.5
0.0
0.0
28.5
0.0
0.5
0.0
57.0
0.0
2.0
0.0
156.5
14.0
1.0
2.0
0.0
6.0
0.0
0.0
1.0
0.5
332.0
18.5
0.0
1.0
1.0
0.0
39.5
1.0
1.0
0.0
10.0
0.0
0.0
0.0
93.0
0.0
0.0
0.0
0.0
2.5
4.0
0.0
1.5
0.5
0.0
158.5
0.0
107.5
3.0
126.5
1.5
0.0
8.5
0.0
7.0
12.0
0.0
24.0
0.0
38.0
12.0
0.5
0.0
1.0
0.0
973.5
0.0
0.0
0.0
11.5
0.0
0.0
15.5
0.0
0.0
0.0
5.5
0.0
0.0
54.0
0.0
0.0
49.5
0.0
1.5
14.0
0.0
31.0
0.0
29.5
23.5
1.5
0.0
5.5
0.0
0.0
0.0
1.5
0.5
275.5
0.0
4.0
2.0
66.0
0.0
21.5
56.0
42.0
1.0
0.0
0.5
46.5
0.0
4.0
0.5
0.0
0.0
3.0
3.0
10.5
0.5
0.0
0.0
1.0
0.5
3.0
9.5
47.5
0.0
4.5
0.0
12.5
0.0
0.0
0.0
39.5
8.5
0.0
0.0
0.0
0.0
287.5
2.5
0.0
122.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
12.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.5
18.0
0.0
0.0
0.0
1.5
62.5
0.0
0.0
0.0
5.0
0.0
1.0
74.0
0.0
0.0
292.5
94.0
21.0
0.0
0.0
0.0
0.0
13.5
13.5
1.0
0.5
0.0
0.0
0.5
45.5
0.0
0.0
0.0
2.5
0.0
0.0
0.0
0.0
0.0
44.5
18.5
2.0
0.0
84.0
34.0
0.0
215.5
0.0
0.0
0.0
3.0
0.0
0.0
1.0

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1426.0
0.0
24.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
6.5
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
180.0
6.0
0.0
1.5
0.0
0.0
1681.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
49.5
1.5
0.0
17.0
0.0
30.0
0.0
0.0
0.0
363.5
526.0
0.0
0.0
0.0
9.5
0.0
5162.0
298.5
0.0
0.0
0.0
0.0
0.0
17.0
1.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
64.0
0.5
0.0
12.0
0.0
1.5
167.5
55.5
814.0
9.0
0.0
0.0
409.5
0.0
0.0
0.0
121.0
0.0
0.0
5.5
0.0
0.0
0.5
0.0
259.5
0.0
0.0
19.0
1.5
0.0
0.0
0.0
6.5
0.0
0.0
1.0
0.0
0.0
0.0
1.5
0.0
0.0
12.0
0.0
0.0
0.0
2.0
0.0
1446.5
0.0
9.5
0.0
0.0
3968.5
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
20.0
0.0
0.0
0.0
0.0
0.0
0.0
167.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
36.0
0.0
0.0
10.0
5572.0
0.0
0.0
511.0
0.0
0.0
0.0
5.0
0.5
25708.5
0.0
0.0
0.0
0.0
0.0
30.0
0.0
0.0
0.0
0.0
4.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
80.5
0.0
0.0
0.0
0.5
0.0
0.0
0.0
1655.0
0.0
29.5
0.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0
0.0
0.0
12.0
6.0
15.0
0.0
330.0
0.5
2705.0
0.0
69.0
0.0
0.0
0.0
0

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
6.5
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.0
0.0
0.0
0.0
0.0
0.0
64.0
0.0
1.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
5.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.0
36.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
184.5
0.0
0.0
0.0
0.0
0.0
23.0
0.0
5.5
0.0
0.0
0.0
19.5
0.0
3.0
0.0
0.0
0.0
0.0
21702.5
0.0
0.0
0.0
0.0
0.0
0.0
41.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0
0.0
0.0
0.0
62.0
0.0
0.0
0.0
0.0
0.0
4.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
30.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
9.0
0.5
0.0
0.0
136.0
0.0
0.0
6.0
2154.5
0.0
0.0
64.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
8.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
2005.0
123.5
0.0
0.0
11.0
0.0
0.0
5.0
0.0
0.0
0.0
0.0
0.0

0.0
27.0
4.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
15.0
0.0
0.0
2657.0
0.0
0.0
0.0
0.0
0.0
39.5
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.5
0.0
0.0
0.0
0.0
0.0
0.0
121.0
0.5
0.0
0.0
0.0
0.5
0.0
0.0
0.0
27.0
0.5
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.0
0.0
0.0
0.0
0.0
325.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
642.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
20.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
21.5
0.0
120.5
0.0
0.0
214.0
3583.5
0.0
0.0
0.0
0.0
0.0
48.0
89.5
0.0
0.0
0.0
0.0
0.0
24.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
81.0
0.0
0.5
866.0
0.0
0.0
0.0
0.0
25.0
0.0
26.5
83.0
0.0
0.0
0.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
6.0
0.0
0.0
0.5
0.0
0.0
11.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
105.5
0.0
0.0
0.0
2.0
0.0
1.0
15.5
0.0
0.0
0.0
17.0
0.5
0.5
0.0
1.0
0.0
133.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
13.0
0.0
2.0
0.0
133161.5
0.0
79.5
21.0
5.5
70

0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
13.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.5
0.5
0.0
0.0
42.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
13.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0

...

0.0
0.0
232.5
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
29.0
0.0
0.0
36.5
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
5.5
0.0
0.0
210.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
42.0
192.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
73.5
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
15.0
0.0
0.0
0.0
0.0
0.0
9.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
806.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.5
14.5
96.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0

62.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
53.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
24.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
2564.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
14.5
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1390.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0
0.0
0.0
4.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
48.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
6.5
0.0
0.0
0.0
0.0
0.0
0.0
8.0
0.0
0.0
0.0
0.0
0.0
0.5
145.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
78596.0
0.0
0.0
0.0
101.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
54.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
134.5
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
66.0
0.0
0.0
22.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
10.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
5.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.5
13.0
0.0
1.0
0.0
528.0
0.0
0.0
0.0
0.0
4.0
0.0
9.0
0.0
0.0
0.0
0.0
0.0
44.5
0.0
0.0
407.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
954.5
674.5
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
5.0
2.0
0.0
0.0
0.5
0.0
40.5
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
7.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
2.0
2.0
0.5
0.0
0.0
426.5
0.0
0.0
0.0
7.5
0.0
0.0
0.0
0.0
0.0
6.5
0.0
76.0
10.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
102631.5
0.0
0.0
0.0
1

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.5
0.0
1.0
0.0
0.0
6.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
85386.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
8.0
0.0
0.0
0.0
114.0
0.0
0.5
0.0
0.0
0.0
0.0
1.0
0.0
0.0
12.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
56.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
64.5
0.0
0.0
1.5
0.0
0.0
0.5
63.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.5
0.0
30.0
0.0
0.0
0.0
19.0
0.0
0.0
0.0
20.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
76.5
0.0
0.0
0.0
0.0
16.5
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.5
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
64463.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
207.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
39.0
0.0
1.0
54.0
4.0
0.0
0.0
2.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
5

7.0
150.5
0.5
0.0
0.0
0.0
0.0
394.5
0.0
0.0
0.0
268.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
13.0
0.0
0.0
0.0
8.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
196.5
0.0
0.0
0.0
4.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.5
0.0
728.0
66.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.5
0.0
0.0
9.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
149.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0
3.0
0.0
30.5
0.0
0.0
0.0
21.5
0.0
0.5
0.0
0.0
0.0
0.0
5.0
0.0
0.0
0.0
13.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
6.5
0.0
120.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
8.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
42.0
0.0
0.0
0.0
0.0
5.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
21.5
14.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
4.5
0.0
0.0
0.0
0.0
0.0
7.0
0.0
127893.5
0.0
295.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
18.5
5.0
3.5
0.0
0.0
0.0
3.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.